In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from sklearn.preprocessing import MultiLabelBinarizer
import itertools

In [2]:
movies = pd.read_csv('movie_bd_v5.csv')
movies.head()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,7.1,2015
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,6.3,2015
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,7.5,2015
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,7.3,2015


# Предобработка

In [3]:
answers = {} # create a dictionary for answers

# The time given in the dataset is in string format, so we need to change it in datetime format
movies['release_date'] = pd.to_datetime(movies['release_date'])

# Add a column with profit
movies['profit'] = movies['revenue'] - movies['budget']

# Add columns where cast, directors, genres and production companies are provided as lists
for field_name in ('cast', 'director', 'genres', 'production_companies'):
    movies[field_name + '_aslist'] = movies[field_name].apply(lambda s: s.split(sep='|'))

# Add a column with months from  movies date of release
movies['release_month'] = movies['release_date'].apply(lambda ts: ts.month)

# Add a column with titles length (number of symbols)
movies['title_length'] = movies['original_title'].apply(len)

# Add a column with overview length (number of words)
movies['overview_length'] = movies['overview'].apply(lambda s: len(s.split(' ')))


# A function creates a dictionary with unique values from some column of strings in a given DataFrame
def build_dict(df, column):
    dict = {}
    for i_line in range(len(df)):
        sublist = df.iloc[i_line][column].split(sep='|')
        for i_item in sublist:
            if not(i_item in dict):
                dict[i_item] = 0
    return dict

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [4]:
answers['1'] = '{} ({})'.format(movies[movies.budget == movies.budget.max()]['original_title'].values[0], \
                                movies[movies.budget == movies.budget.max()].imdb_id.values[0])
print(answers['1'])
# + right answer

Pirates of the Caribbean: On Stranger Tides (tt1298650)


# 2. Какой из фильмов самый длительный (в минутах)?

In [5]:
answers['2'] = '{} ({})'.format(movies[movies.runtime == movies.runtime.max()]['original_title'].values[0], \
                                movies[movies.runtime == movies.runtime.max()].imdb_id.values[0])
print(answers['2'])
# + right answer

Gods and Generals (tt0279111)


# 3. Какой из фильмов самый короткий (в минутах)?





In [6]:
answers['3'] = '{} ({})'.format(movies[movies.runtime == movies.runtime.min()]['original_title'].values[0], \
                                movies[movies.runtime == movies.runtime.min()].imdb_id.values[0])
print(answers['3'])
# + right answer

Winnie the Pooh (tt1449283)


# 4. Какова средняя длительность фильмов?


In [7]:
answers['4'] = int(round(movies.runtime.mean(), 0))
print(answers['4'])
# + right answer

110


# 5. Каково медианное значение длительности фильмов? 

In [8]:
answers['5'] = int(round(movies.runtime.median(), 0))
print(answers['5'])
# + right answer

107


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [9]:
answers['6'] = '{} ({})'.format(movies[movies.profit == movies.profit.max()]['original_title'].values[0], \
                                movies[movies.profit == movies.profit.max()].imdb_id.values[0])
print(answers['6'])
# + right answer

Avatar (tt0499549)


# 7. Какой фильм самый убыточный? 

In [10]:
answers['7'] = '{} ({})'.format(movies[movies.profit == movies.profit.min()]['original_title'].values[0], \
                                movies[movies.profit == movies.profit.min()].imdb_id.values[0])
print(answers['7'])
# + right answer

The Lone Ranger (tt1210819)


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [11]:
answers['8'] = len(movies[movies.profit > 0])
print(answers['8'])
# + right answer

1478


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [12]:
answers['9'] = '{} ({})'.format(movies[(movies.revenue == movies[movies.release_year == 2008].revenue.max()) & (movies.release_year == 2008)]['original_title'].values[0], \
                                movies[(movies.revenue == movies[movies.release_year == 2008].revenue.max()) & (movies.release_year == 2008)].imdb_id.values[0])
print(answers['9'])
# + right answer

The Dark Knight (tt0468569)


ВАРИАНТ 2

In [13]:
start_date = pd.Timestamp(day=1, month=1, year=2008)
end_date = pd.Timestamp(day=31, month=12, year=2008)
print('{} ({})'.format(movies[movies.revenue == movies[movies.release_date.between(start_date, end_date)].revenue.max()]['original_title'].values[0], \
                       movies[movies.revenue == movies[movies.release_date.between(start_date, end_date)].revenue.max()].imdb_id.values[0]))

The Dark Knight (tt0468569)


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [14]:
start_date = pd.Timestamp(day=1, month=1, year=2012)
end_date = pd.Timestamp(day=31, month=12, year=2014)
answers['10'] = '{} ({})'.format(movies[movies.profit == movies[movies.release_date.between(start_date, end_date)].profit.min()]['original_title'].values[0], \
                                 movies[movies.profit == movies[movies.release_date.between(start_date, end_date)].profit.min()].imdb_id.values[0])
print(answers['10'])
# + right answer

The Lone Ranger (tt1210819)


ВАРИАНТ 2

In [15]:
'{} ({})'.format(movies[(movies['profit'] == movies[movies['release_year'].isin([2012, 2013, 2014])]['profit'].min()) & (movies['release_year'].isin([2012, 2013, 2014]))]['original_title'].values[0], \
                 movies[(movies['profit'] == movies[movies['release_year'].isin([2012, 2013, 2014])]['profit'].min()) & (movies['release_year'].isin([2012, 2013, 2014]))].imdb_id.values[0])

'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

ВАРИАНТ 1

In [16]:
%%time

# Create a list with all genres from dataframe and count how many times each of them is presented in the list by Counter
genres_count_1 = Counter(movies['genres'].str.cat(sep='|').split(sep='|')) 
answers['11'] = pd.Series(genres_count_1).sort_values(ascending=False).index[0]
print(answers['11'])
# + right answer

Drama
Wall time: 0 ns


ВАРИАНТ 2

In [17]:
%%time

# Explode dataframe by genres: duplicate lines with multiple genres in order each line to have only 1 genre
genres_count_2 = movies.explode(column='genres_aslist')['genres_aslist'].value_counts()

print('{}: {} entries'.format(pd.Series(genres_count_2).sort_values(ascending=False).index[0], \
                              pd.Series(genres_count_2).sort_values(ascending=False)[0]))

Drama: 782 entries
Wall time: 26.8 ms


ВАРИАНТ 3

In [18]:
%%time

# Build a dictionary with all unique genres
genres_count_3 = build_dict(movies, 'genres')

# For each genre in the dictionary count how often it's mentioned in the data frame
for key in genres_count_3.keys():
    genres_count_3[key] = movies['genres'].str.contains(key).sum()

print('{}: {} entries'.format(pd.Series(genres_count_3).sort_values(ascending=False).index[0], \
                              pd.Series(genres_count_3).sort_values(ascending=False)[0]))

Drama: 782 entries
Wall time: 268 ms


ВАРИАНТ 4

In [19]:
%%time

genres_count_4 = defaultdict(int)

for i_line in range(len(movies)): # for each line in the DataFrame
    for i_item in movies.iloc[i_line]['genres_aslist']: # count each item in every list of genres
        genres_count_4[i_item] += 1

print('{}: {} entries'.format(pd.Series(genres_count_4).sort_values(ascending=False).index[0], \
                              pd.Series(genres_count_4).sort_values(ascending=False)[0]))

Drama: 782 entries
Wall time: 240 ms


ВАРИАНТ 5

In [20]:
%%time

# Optimize the variant 4:
# In order not to go through the whole dataframe, let's create a series with unique sets of genres
# Number of unique sets of genres is less than the number of lines in the dataframe
genres_series = movies.genres_aslist.value_counts() # series of lists

genres_count_5 = Counter()
for i in range(len(genres_series)):
    for genre in genres_series.index[i]:
        genres_count_5[genre] += genres_series.iloc[i]

print('{}: {} entries'.format(pd.Series(genres_count_5).sort_values(ascending=False).index[0], \
                              pd.Series(genres_count_5).sort_values(ascending=False)[0]))

Drama: 782 entries
Wall time: 50 ms


ВАРИАНТ 6

In [21]:
%%time

# Make a copy of the movies dataset binary coded for each genre
mlb = MultiLabelBinarizer() # use multi label binarizer from sklearn library
movies_bicoded_genres = movies.copy()
movies_bicoded_genres = movies_bicoded_genres.join(pd.DataFrame(mlb.fit_transform(movies_bicoded_genres.pop('genres_aslist')), columns=mlb.classes_, index=movies_bicoded_genres.index))

# For each genre create a separate column
# Each movie has 1 in the colums if it belongs to the genre and 0 if not
sums_per_genres = movies_bicoded_genres.loc[:, 'Action':].sum() # sum values per each columns with genres

print('{}: {} entries'.format(sums_per_genres[sums_per_genres == sums_per_genres.max()].index[0], \
                              sums_per_genres.max()))

Drama: 782 entries
Wall time: 65 ms


First two algorithms and binary coding has lower time to run

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

ВАРИАНТ 1

In [22]:
%%time

# Create a subset of profitable movies, create a list with all genres from this subset and count how many times each of them is presented in the list by Counter
answers['12'] = pd.Series(Counter(movies[movies['profit'] > 0]['genres'].str.cat(sep='|').split(sep='|'))).sort_values(ascending=False).index[0]
print(answers['12'])
# + right answer

Drama
Wall time: 5.1 ms


ВАРИАНТ 2

In [23]:
%%time

# Take a subset with only profitable movies and explode it by genres: duplicate lines with multiple genres in order each line to have only 1
profit_movies_bygenre_1 = movies[movies['profit'] > 0].explode(column='genres_aslist')['genres_aslist'].value_counts().sort_values(ascending=False)

print('{}: {} profitable movies'.format(profit_movies_bygenre_1.index[0], profit_movies_bygenre_1[0]))

Drama: 560 profitable movies
Wall time: 19.9 ms


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [24]:
%%time

# Explode the dataframe by directors: duplicate lines with multiple directos in order each line to have only 1
# Sum revenue per each director
answers['13'] = movies.explode(column='director_aslist').groupby('director_aslist')['revenue'].sum().sort_values(ascending=False).index[0]
print(answers['13'])
# + right answer

Peter Jackson
Wall time: 19.7 ms


# 14. Какой режисер снял больше всего фильмов в стиле Action?

ВАРИАНТ 1

In [25]:
%%time

# Create a subset with action movies, create a list with all directors from this subset and count how many times each of them is presented in the list by Counter
answers['14'] = pd.Series(Counter(movies[movies['genres'].str.contains('Action')]['director'].str.cat(sep='|').split(sep='|'))).sort_values(ascending=False).index[0]
print(answers['14'])
# + right answer

Robert Rodriguez
Wall time: 9.63 ms


ВАРИАНТ 2

In [26]:
%%time

# Take a subset with only action movies and explode it by directors: duplicate lines with multiple directors in order each line to have only 1
# Count number of movies per each director
action_bydirector_1 = movies[movies['genres'].str.contains('Action')].explode(column='director_aslist')['director_aslist'].value_counts().sort_values(ascending=False)

print('{}: {} action movies'.format(action_bydirector_1.index[0], action_bydirector_1[0]))

Robert Rodriguez: 9 action movies
Wall time: 14.8 ms


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [27]:
%%time

# Take a subset with movies released in 2012 and explode it by actors: duplicate lines with multiple actors in order each line to have only 1
# Sum revenue per each actor
answers['15'] = movies[movies['release_year'] == 2012].explode(column='cast_aslist').groupby('cast_aslist')['revenue'].sum().sort_values(ascending=False).index[0]
print(answers['15'])
# + right answer

Chris Hemsworth
Wall time: 11.2 ms


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

ВАРИАНТ 1

In [28]:
%%time

# Create a subset with expensive movies (budget of a movie is higher than average)
# Create a list with all actors from this subset and count how many times each of them is presented in the list by Counter
answers['16'] = pd.Series(Counter(movies[movies['budget'] > movies['budget'].mean()]['cast'].str.cat(sep='|').split(sep='|'))).sort_values(ascending=False).index[0]
print(answers['16'])
# + right answer

Matt Damon
Wall time: 9.73 ms


ВАРИАНТ 2

In [29]:
%%time

# Take a subset with expensive movies and explode it by actors: duplicate lines with multiple actors in order each line to have only 1
# Count number of movies per each actor
exp_movies_byactor_1 = movies[movies['budget'] > movies['budget'].mean()].explode(column='cast_aslist')['cast_aslist'].value_counts().sort_values(ascending=False)

print('{}: {} expensive movies'.format(exp_movies_byactor_1.index[0], exp_movies_byactor_1[0]))

Matt Damon: 18 expensive movies
Wall time: 20.2 ms


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

ВАРИАНТ 1

In [30]:
%%time

# Create a subset with Nicolas Cage movies
# Create a list with all genres from this subset and count how many times each of them is presented in the list by Counter
answers['17'] = pd.Series(Counter(movies[movies['cast'].str.contains('Nicolas Cage')]['genres'].str.cat(sep='|').split(sep='|'))).sort_values(ascending=False).index[0]
print(answers['17'])
# + right answer

Action
Wall time: 9.32 ms


ВАРИАНТ 2

In [31]:
%%time

# Take a subset with Nicolas Cage movies and explode it by genres: duplicate lines with multiple genres in order each line to have only 1
# Count number of movies per each genre
NCage_movies_bygenre_1 = movies[movies['cast'].str.contains('Nicolas Cage')].explode(column='genres_aslist')['genres_aslist'].value_counts().sort_values(ascending=False)

print('{}: {} movies with Nicolas Cage'.format(NCage_movies_bygenre_1.index[0], NCage_movies_bygenre_1[0]))

Action: 17 movies with Nicolas Cage
Wall time: 9.57 ms


# 18. Самый убыточный фильм от Paramount Pictures

In [32]:
loss_paramount = movies[(movies['profit'] == movies[movies['production_companies'].str.contains('Paramount Pictures')]['profit'].min()) & \
                        (movies['production_companies'].str.contains('Paramount Pictures'))]
answers['18'] = '{} ({})'.format(loss_paramount['original_title'].values[0], \
                                 loss_paramount['imdb_id'].values[0])
print(answers['18'])
# + right answer

K-19: The Widowmaker (tt0267626)


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [33]:
revenue_by_year = movies.groupby('release_year')['revenue'].sum().sort_values(ascending=False)
answers['19'] = revenue_by_year.index[0]
print(answers['19'])
# + right answer

2015


# 20. Какой самый прибыльный год для студии Warner Bros?

In [34]:
profit_by_year_WB = movies[movies.production_companies.str.contains('Warner Bros')].groupby('release_year')['profit'].sum().sort_values(ascending=False)
answers['20'] = profit_by_year_WB.index[0]
print(answers['20'])
# + right answer

2014


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [35]:
count_bymonth = movies.groupby('release_month')['imdb_id'].count().sort_values(ascending=False)
answers['21'] = count_bymonth.index[0]
print(answers['21'])
# + right answer

9


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [36]:
answers['22'] = movies.groupby('release_month')['imdb_id'].count()[5:8].sum()
print(answers['22'])
# + right answer

450


ВАРИАНТ 2

In [37]:
len(movies[movies['release_month'].isin([6, 7, 8])])

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

ВАРИАНТ 1

In [38]:
%%time

# Filter only movies released in winter
# For these movies create a list with all directors and count how many times each of them is presented in the list by Counter
# Sort series with directors and their number of movies released in winter and take the highest as the answer
winter_movies_bydirector_1 = pd.Series(Counter(movies[movies['release_month'].isin([1, 2, 12])]['director'].str.cat(sep='|').split(sep='|'))).sort_values(ascending=False)
answers['23'] = winter_movies_bydirector_1.index[0]
print(answers['23'])
# + right answer

Peter Jackson
Wall time: 4.51 ms


ВАРИАНТ 2

In [39]:
%%time

# Explode the dataframe with movies released in winter by directors: duplicate lines with multiple directors in order each line to have only 1
movies_exploded_bydirector = movies[movies['release_month'].isin([1, 2, 12])].explode(column='director_aslist')

print('{}: {} movies released in winter'.format(movies_exploded_bydirector['director_aslist'].value_counts().index[0], \
                                                movies_exploded_bydirector['director_aslist'].value_counts()[0]))

Peter Jackson: 7 movies released in winter
Wall time: 11.2 ms


ВАРИАНТ 3

In [40]:
%%time

# Make a subset of movies released in winter and build a dictionary with all unique directors in this subset
winter_movies = movies[movies['release_month'].isin([1, 2, 12])].copy()
winter_movies_bydirector_2 = build_dict(winter_movies, 'director')

# For each director in the dictionary count the number of movies
for key in winter_movies_bydirector_2.keys():
    winter_movies_bydirector_2[key] = winter_movies['director'].str.contains(key).sum()

print('{}: {} movies released in winter'.format(pd.Series(winter_movies_bydirector_2).sort_values(ascending=False).index[0], \
                                                pd.Series(winter_movies_bydirector_2).sort_values(ascending=False)[0]))

Peter Jackson: 7 movies released in winter
Wall time: 210 ms


ВАРИАНТ 4

In [41]:
%%time

winter_movies_bydirector_3 = defaultdict(int)

for i_line in range(len(winter_movies)): # for each of the line in the subset of movies released in winter
    for i_item in winter_movies.iloc[i_line]['director_aslist']: # count each item in every list of directors
        winter_movies_bydirector_3[i_item] += 1

print('{}: {} movies released in winter'.format(pd.Series(winter_movies_bydirector_3).sort_values(ascending=False).index[0], \
                                                pd.Series(winter_movies_bydirector_3).sort_values(ascending=False)[0]))

Peter Jackson: 7 movies released in winter
Wall time: 65.4 ms


ВАРИАНТ 5

In [42]:
%%time

# Make a copy of the dataset with movies released in winter binary coded for each director
winter_movies_bicoded_director = winter_movies.copy()
winter_movies_bicoded_director = winter_movies_bicoded_director.join(pd.DataFrame(mlb.fit_transform(winter_movies_bicoded_director.pop('director_aslist')), columns=mlb.classes_, index=winter_movies_bicoded_director.index))

# For each director create a separate column
# Each movie has 1 in the colums if it was filmed by the director to the genre and 0 if not
winter_movies_sum_bydirector = winter_movies_bicoded_director.loc[:, 'Aaron Seltzer':].sum() # sum values per each columns with directors
print('{}: {} movies released in winter'.format(winter_movies_sum_bydirector[winter_movies_sum_bydirector == winter_movies_sum_bydirector.max()].index[0], \
                                                winter_movies_sum_bydirector.max()))

Peter Jackson: 7 movies released in winter
Wall time: 9.55 ms


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [43]:
# Explode dataframe by production companies: duplicate lines with multiple companies in order each line to have only 1
movies_exploded_bycompanies = movies.explode(column='production_companies_aslist')

# Group average title length per production company
answers['24'] = movies_exploded_bycompanies.groupby('production_companies_aslist')['title_length'].mean().sort_values(ascending=False).index[0]
print(answers['24'])
# + right answer

Four By Two Productions


ВАРИАНТ 2

In [44]:
# Build a dictionary with all unique production companies
title_length_bystudio = build_dict(movies, 'production_companies')

# For each production company in the dictionary find the average title length
for key in title_length_bystudio.keys():
    title_length_bystudio[key] = movies[movies['production_companies'].str.contains(key)]['title_length'].mean()

print('{}: {} symbols per title in average'.format(pd.Series(title_length_bystudio).sort_values(ascending=False).index[0], \
                                                   pd.Series(title_length_bystudio).sort_values(ascending=False)[0]))

C:\Users\Lenovo\anaconda3\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Four By Two Productions: 83.0 symbols per title in average


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [45]:
# From the dataframe exploded by companies (from the previous question)
# Group average overview length per production company
answers['25'] = movies_exploded_bycompanies.groupby('production_companies_aslist')['overview_length'].mean().sort_values(ascending=False).index[0]
print(answers['25'])
# + right answer

Midnight Picture Show


ВАРИАНТ 2

In [46]:
# Build a dictionary with all unique production companies
overview_length_bystudio = build_dict(movies, 'production_companies')

# For each production company in the dictionary find the average overview length
for key in overview_length_bystudio.keys():
    overview_length_bystudio[key] = movies[movies['production_companies'].str.contains(key)]['overview_length'].mean()

print('{}: {} words per overview in average'.format(pd.Series(overview_length_bystudio).sort_values(ascending=False).index[0], \
                                                    pd.Series(overview_length_bystudio).sort_values(ascending=False)[0]))

Midnight Picture Show: 175.0 words per overview in average


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [47]:
answers['26'] = movies[movies['vote_average'] > movies['vote_average'].quantile(0.99)]['original_title'].values
print(answers['26'])
# + right answer

['Inside Out' 'Room' 'Interstellar' 'Guardians of the Galaxy'
 'The Imitation Game' 'Gone Girl' 'The Grand Budapest Hotel' 'Inception'
 'The Dark Knight' 'The Pianist'
 'The Lord of the Rings: The Return of the King' 'The Wolf of Wall Street'
 '12 Years a Slave' 'Memento']


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [48]:
# Create an empty list for all pairs of actors existing in the data frame
pairs_list = []

for i_line in range(len(movies)):
    actors_pairs_sublist = list(itertools.combinations(movies.cast_aslist.iloc[i_line], 2)) # for each line of dataframe create a sublist with all actors combinations by 2
    for pair in actors_pairs_sublist:
        pairs_list.append(tuple(sorted(pair))) # sort each pair in order to avoid ('a', 'b') != ('b', 'a') and add it to the common list

pairs_counts = pd.Series(Counter(pairs_list)) # count the number of entries for each pair
answers['27'] = pairs_counts.sort_values(ascending = False).index[0:3].values # top three pairs
print(answers['27'])
# + right answer

[('Daniel Radcliffe', 'Emma Watson') ('Daniel Radcliffe', 'Rupert Grint')
 ('Emma Watson', 'Rupert Grint')]


# Submission

In [49]:
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 9,
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': array(['Inside Out', 'Room', 'Interstellar', 'Guardians of the Galaxy',
        'The Imitation Game', 'Gone Girl', 'The Grand Budapest Hotel',
        'Inception', 'The Dark Knight', 'The Pianist',
        'The Lord of the Rings: The Return of the King',
        'The Wolf of Wall Street', '12 Years a Slave', 'Memento'],
       dtype=object),
 '27':

In [50]:
len(answers)

27